In [2]:
# Clear cache of spark cluster
from pyspark.shell import spark
spark.catalog.clearCache()
print(spark.version)


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.0.0
      /_/

Using Python version 3.8.5 (default, Sep  4 2020 07:30:14)
SparkSession available as 'spark'.
3.0.0


In [ ]:
import pandas as pd
from pyspark.sql import functions as f
from pyspark.sql.window import Window


In [ ]:
#pd_df=pd.read_json("https://data.rivm.nl/covid-19/COVID-19_casus_landelijk.json")
#df = spark.createDataFrame(pd_df)



In [ ]:
#df.groupBy(df.Date_statistics).count().orderBy(df.Date_statistics, ascending=False).head(n=7)



In [ ]:
pd_df=pd.read_json("https://data.rivm.nl/covid-19/COVID-19_aantallen_gemeente_cumulatief.json")
df = spark.createDataFrame(pd_df).withColumn("Date_of_report",f.to_timestamp(f.col("Date_of_report"),"yyyy-MM-dd HH:mm:ss"))




In [ ]:
icol = ["Total_reported","Hospital_admission","Deceased"]
df = df.groupby('Date_of_report').agg(*[f.sum(x).alias(x+"_by_date") for x in icol])

In [ ]:
window=Window.partitionBy().orderBy("Date_of_report")
icol_new=[col +"_by_date" for col in icol]
 
for col in icol_new:
  df=df.withColumn(col+"_diff",f.col(col)-f.lag(col,1).over(window))

df=df.orderBy("Date_of_report", ascending=False)

In [ ]:
days = lambda i: i * 86400 

In [ ]:
window_7_day=Window.partitionBy().orderBy(f.col("Date_of_report").cast("date")).rowsBetween(-6,0)
icol_new_7_day_average=[col +"_diff" for col in icol_new]
df_temp=df
for col in icol_new_7_day_average:
  print(col)
  df_temp = df_temp.withColumn(col+"_7_day_average",f.avg(col).over(window_7_day))
  
df=df_temp


In [ ]:
print(df.select("Date_of_report","Total_reported_by_date_diff","Total_reported_by_date_diff_7_day_average").orderBy("Date_of_report", ascending=False).head(n=7))

In [ ]:
print(df.select("Date_of_report","Hospital_admission_by_date_diff","Hospital_admission_by_date_diff_7_day_average").orderBy("Date_of_report", ascending=False).head(n=3))

In [ ]:
display(df.select("Date_of_report","Total_reported_by_date_diff","Total_reported_by_date_diff_7_day_average").orderBy("Date_of_report"))

In [ ]:
display(df.select("Date_of_report",'Total_reported_by_date_diff').orderBy("Date_of_report"))

In [ ]:
display(df.select("Date_of_report","Hospital_admission_by_date_diff_7_day_average").orderBy("Date_of_report"))